In [1]:
import pandas as pd
# import numpy as np
# from sklearn.pipeline import Pipeline
# from sklearn.impute import SimpleImputer
# from sklearn.preprocessing import OneHotEncoder, StandardScaler
# from sklearn.compose import ColumnTransformer

from pyspark.sql import SparkSession
import os

In [14]:
label_name = 'Hráč vyhrál'

In [6]:
spark = SparkSession.builder.appName("elo_app").getOrCreate();

In [15]:
df = spark.read.csv('data/2_raw_unioned.csv.gz', header=True)
df.limit(5).toPandas()

,Hráč,Oddíl,Družstvo,Soupeř,Oddíl soupeř,Družstvo soupeř,Výsledek,Datum zápasu,Soutěž,Elo hráče,...,Elo (max),Elo soupeře,Elo (min) soupeře,Elo (max) soupeře,Rok narození,Max elo,Elo nejlepšího poraženého soupeře,ID Hráč,ID Soupeř,Rok narození soupeř
0,Tomáš Tregler,"HB Ostrov , z.s.",HB Ostrov H.Brod,Tomáš Martinko,Tělovýchovná jednota Ostrava,TJ Ostrava KST,0:3,26.11.2021,Extraliga mužů,2454-> 2444,...,2482-> 2470,2306-> 2318,2289-> 2301,2327-> 2338,1990,2458,2446,/st/hrac/tregler_tomas_1990,martinko_tomas_2002,2002
1,Tomáš Tregler,"HB Ostrov , z.s.",HB Ostrov H.Brod,Patrik Klos,Tělovýchovná jednota Ostrava,TJ Ostrava KST,3:1,26.11.2021,Extraliga mužů,2452-> 2454,...,2480-> 2482,2317-> 2315,2300-> 2299,2343-> 2341,1990,2458,2446,/st/hrac/tregler_tomas_1990,klos_patrik_1998,1998
2,Tomáš Tregler,"HB Ostrov , z.s.",HB Ostrov H.Brod,David Reitšpies,SKST Cheb,SKST Cheb,1:3,11.11.2021,Extraliga mužů,2458-> 2452,...,2485-> 2479,2409-> 2419,2356-> 2380,2476-> 2484,1990,2458,2446,/st/hrac/tregler_tomas_1990,reitspies_david_1996,1996
3,Tomáš Tregler,"HB Ostrov , z.s.",HB Ostrov H.Brod,Stanislav Kučera,SKST Cheb,SKST Cheb,3:0,11.11.2021,Extraliga mužů,2454-> 2458,...,2482-> 2485,2380-> 2377,2339-> 2337,2407-> 2403,1990,2458,2446,/st/hrac/tregler_tomas_1990,kucera_stanislav_1997,1997
4,Tomáš Tregler,"HB Ostrov , z.s.",HB Ostrov H.Brod,Martin Koblížek,Klub stolního tenisu KT Praha,KT Praha,3:0,5.11.2021,Extraliga mužů,2453-> 2454,...,2482-> 2482,2241-> 2241,2215-> 2215,2264-> 2263,1990,2458,2446,/st/hrac/tregler_tomas_1990,koblizek_martin_1996,1996


In [16]:
from datetime import datetime as dt

def get_season(dt):
    if dt.month >= 3 and dt.month <= 5:
        return 1
    if dt.month >= 6 and dt.month <= 8:
        return 2
    if dt.month >= 9 and dt.month <= 11:
        return 3
    return 4

def petisetovka(hrac, souper):
    hrac = int(hrac)
    souper = int(souper)
    if (hrac < 2) or (souper < 2): return None
    if hrac > souper: return True
    return False

In [53]:
df['Hráč sety'] = df['Výsledek'].str[0]
df['Soupeř sety'] = df['Výsledek'].str[2]
df['Hráč vyhrál'] = df['Hráč sety'] > df['Soupeř sety']
df['Datum zápasu'] = pd.to_datetime(df['Datum zápasu'])
df['Den v týdnu (číslo)'] = df['Datum zápasu'].dt.weekday + 1
df['Elo hráče před'] = df['Elo hráče'].str.split('->', expand=True)[0]
df['Elo hráče po'] = df['Elo hráče'].str.split('->', expand=True)[1]
df['Elo soupeře před'] = df['Elo soupeře'].str.split('->', expand=True)[0]
df['Soupeř je žena'] = df['Soupeř'].apply(lambda x: False if re.search(".*ová$", x) is None else True)
df['Hráč je žena'] = df['Hráč'].apply(lambda x: False if re.search(".*ová$", x) is None else True)
df['Datum zápasu'] = pd.to_datetime(df['Datum zápasu'])
df['isWeekend'] = df['Den v týdnu (číslo)'] >= 6
df['obdobi'] = df['Datum zápasu'].apply(lambda x: get_season(x))
X = df[['Rok narození', 'Elo hráče před', 'Hráč je žena',
        'Rok narození soupeř', 'Elo soupeře před', 'Soupeř je žena',
        'Den v týdnu (číslo)', 'isWeekend', 'obdobi']]
y = df[label_name]

X

,Rok narození,Elo hráče před,Hráč je žena,Rok narození soupeř,Elo soupeře před,Soupeř je žena,Den v týdnu (číslo),isWeekend,obdobi
0,1976,1929,False,1968,1770,False,6,True,3
1,1976,1929,False,1975,1646,False,6,True,3
2,1976,1929,False,1964,1523,False,6,True,3
3,1976,1928,False,1995,1728,False,6,True,3
4,1976,1928,False,1953,1540,False,6,True,3
...,...,...,...,...,...,...,...,...,...
8232,1964,944,False,1971,1109,False,1,False,4
8233,1964,944,False,2000,1,False,4,False,1
8234,1964,950,False,1975,909,False,4,False,1
8235,1964,950,False,1967,1,False,4,False,1


In [54]:
ss = StandardScaler()
scaled = ss.fit_transform(X)
scaled_df = pd.DataFrame(scaled)
scaled_df.columns = X.columns
scaled_df

array([[-6.72357426e-04,  2.48657274e+00,  0.00000000e+00, ...,
         5.97240556e-01,  1.12711885e+00,  8.63033980e-02],
       [-6.72357426e-04,  2.48657274e+00,  0.00000000e+00, ...,
         5.97240556e-01,  1.12711885e+00,  8.63033980e-02],
       [-6.72357426e-04,  2.48657274e+00,  0.00000000e+00, ...,
         5.97240556e-01,  1.12711885e+00,  8.63033980e-02],
       ...,
       [-6.39696371e-01, -2.52890377e+00,  0.00000000e+00, ...,
        -5.27852745e-01, -8.87217879e-01, -1.84806691e+00],
       [-6.39696371e-01, -2.52890377e+00,  0.00000000e+00, ...,
        -5.27852745e-01, -8.87217879e-01, -1.84806691e+00],
       [-6.39696371e-01, -2.53914989e+00,  0.00000000e+00, ...,
        -5.27852745e-01, -8.87217879e-01, -1.84806691e+00]])

In [58]:
scaled_df.to_csv('data/3_X.csv', index=False)
y.to_csv('data/3_y.csv', index=False)